In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import math
import random
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, roc_curve
from statannot import add_stat_annotation
import seaborn as sns
import time

import networkx as nx
from tqdm import tqdm
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from dgl.nn.pytorch import GraphConv, SAGEConv, TAGConv, GINConv

from deepsurv_utils import c_index, adjust_learning_rate
# from loss import NegativeLogLikelihood

Using backend: pytorch


In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# load SHPH data
all_patient_info = pd.read_csv("/home/jielian/lung-graph-project/data/csv/SPH0812.csv")
all_patient_info = all_patient_info[['folder_name', 'Sex_1_male_2_female', 'Age',
       'Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL','Histology_1_Adenocarcinoma_2_SquamousCellCarcinoma_3_Others',
        'pT_Stage', 'pN_Stage', 'pM_Stage', 'pTNM', 'RFS_Status', 'RFS_Month',
       'OS_Status', 'OS_Month']]
stage1 = list(np.load("/home/jielian/lung-graph-project/data/seg_image/labels/name_stage1.npy"))
stage2 = list(np.load("/home/jielian/lung-graph-project/data/seg_image/labels/name_stage2.npy"))
patint_list = [*stage1, *stage2]
patient_info = all_patient_info[all_patient_info['folder_name'].isin(patint_list)]

patient_info['pT_Stage']=patient_info['pT_Stage'].replace({"T1a":0, "T1b":0, "T1c":0, "T2a":1,"T2b":1,"T3":2})
patient_info['pM_Stage']=patient_info['pM_Stage'].replace({"M1a":1})

feature_files = os.listdir("trans_feature")

/tmp/ipykernel_6264/940870848.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_info['pT_Stage']=patient_info['pT_Stage'].replace({"T1a":0, "T1b":0, "T1c":0, "T2a":1,"T2b":1,"T3":2})
/tmp/ipykernel_6264/940870848.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_info['pM_Stage']=patient_info['pM_Stage'].replace({"M1a":1})


In [3]:
trans_feature = "/home/jielian/lung-graph-project/data/seg_image/medicalnet/tumor_medical/"

data = []
name = []
for feature_name in feature_files:
#     print(feature_name)
    path = trans_feature +feature_name[:-4] + "_d64.npy"
    name.append(int(feature_name[:-4]))
    feature = np.load(path, allow_pickle=True)
    af = feature.reshape((-1, 8, 8, 8))
    af_mean = af.mean(axis=(1, 2, 3))
    data.append(af_mean)
feature_data = pd.DataFrame(data)
feature_data = (feature_data-feature_data.min())/(feature_data.max()-feature_data.min())
feature_data['folder_name']=name
all_data = patient_info.merge(feature_data, how='left', on='folder_name')
print(len(all_data))

1705


In [4]:
all_data['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'] = all_data['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'].replace({4:3})

In [5]:
# load external information
external_info = pd.read_csv("/home/jielian/lung-graph-project/Tumor_tranformer/data_ind/External_label.csv")
external_patint_list = external_info['Patient']
external_info=external_info.rename(columns={"Patient":"folder_name"})
external_info=external_info.rename(columns={"Histology":"Histology_1_Adenocarcinoma_2_SquamousCellCarcinoma_3_Others"})
external_info['pT_Stage']=external_info['pT_Stage'].replace({"Tis":0,"T1a":0, "T1b":0, "T1c":0, "T2a":1,"T2b":1,"T3":2,"T4":3 })


In [6]:
external_feature_files = os.listdir("trans_feature_val")
external_info['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'] = external_info['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'].replace({4:3})
external_data = []
for feature_name in external_patint_list:
    path = "/home/jielian/lung-graph-project/data/seg_image/External_medicalnet/tumor/"+feature_name+"_d64.npy"
    feature = np.load(path, allow_pickle=True)
    af = feature.reshape((-1, 8, 8, 8))
    af_mean = af.mean(axis=(1, 2, 3))
    external_data.append(af_mean)
external_feature_data = pd.DataFrame(external_data)
external_feature_data = (external_feature_data-external_feature_data.min())/(external_feature_data.max()-external_feature_data.min())
external_feature_data['folder_name']=external_patint_list
external_all_data = external_info.merge(external_feature_data, how='left', on='folder_name')
print(len(external_all_data))

159


In [7]:
# # #merge the dataset
frames = [all_data, external_all_data]
final_data = pd.concat(frames)

In [8]:
train_id = np.load("data_ind/train_index.npy",allow_pickle=True)
val_id = np.load("data_ind/val_index.npy", allow_pickle=True)
test_id = np.load("data_ind/test_index.npy",allow_pickle=True)
external_id = np.array(range(len(all_data),len(final_data)))
idx_train = torch.LongTensor(train_id)
idx_val = torch.LongTensor(val_id)
idx_test = torch.LongTensor(test_id)
idx_external_val = torch.LongTensor(external_id)

print("training OS distribution:")
print(all_data.iloc[train_id,:]['OS_Status'].value_counts())
print("validation OS distribution:")
print(all_data.iloc[val_id,:]['OS_Status'].value_counts())
print("test OS distribution:")
print(all_data.iloc[test_id,:]['OS_Status'].value_counts())
print("External OS distribution:")
print(final_data.iloc[external_id,:]['OS_Status'].value_counts())


print("training RFS_Status distribution:")
print(all_data.iloc[train_id,:]['RFS_Status'].value_counts())
print("validation RFS_Status distribution:")
print(all_data.iloc[val_id,:]['RFS_Status'].value_counts())
print("test RFS_Status distribution:")
print(all_data.iloc[test_id,:]['RFS_Status'].value_counts())
print("External RFS_Status distribution:")
print(final_data.iloc[external_id,:]['RFS_Status'].value_counts())

training OS distribution:
0    991
1    287
Name: OS_Status, dtype: int64
validation OS distribution:
0    173
1     41
Name: OS_Status, dtype: int64
test OS distribution:
0    169
1     44
Name: OS_Status, dtype: int64
External OS distribution:
0    102
1     57
Name: OS_Status, dtype: int64
training RFS_Status distribution:
0    931
1    347
Name: RFS_Status, dtype: int64
validation RFS_Status distribution:
0    158
1     56
Name: RFS_Status, dtype: int64
test RFS_Status distribution:
0    154
1     59
Name: RFS_Status, dtype: int64
External RFS_Status distribution:
0    113
1     46
Name: RFS_Status, dtype: int64


# Start Graph Building!!

In [9]:
# define similarity of two patient
def SimScore(a1,a2,s1,s2,l1,l2,h1,h2,t1,t2,n1,n2,m1,m2,tnm1,tnm2): 
    c_score = 0
    h_score = 0
    t_score = 0
    l_score = 0
    # sex and age
    if s1 == s2:
        c_score +=1
    if abs(a1-a2) <= 5:
        c_score +=1
    
    if l1 == l2:
        l_score +=1
        
    if h1 == h2:
        h_score +=1
    
#     if (t1 == t2) and (n1 == n2) and (m1 == m2):
#         t_score +=1
    
    if t1 == t2:
        t_score +=1
    if n1 == n2:
        t_score +=1
    if m1 == m2:
        t_score +=1



    return c_score*t_score*h_score*l_score


def adj_matrix(patient_info):
    age = patient_info['Age'].to_list()
    sex = patient_info['Sex_1_male_2_female'].to_list()
    loc = patient_info['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'].to_list()
    his = patient_info['Histology_1_Adenocarcinoma_2_SquamousCellCarcinoma_3_Others'].to_list()
    pts = patient_info['pT_Stage'].to_list()
    pns = patient_info['pN_Stage'].to_list()
    pms = patient_info['pM_Stage'].to_list()
    tnm = patient_info['pTNM'].to_list()

    edge_list=[]
    edge_wight=[]
    n_sample = len(age)
    adj = np.zeros((n_sample, n_sample))
    for i in range(n_sample):
        for j in range(n_sample):
            adj[i,j] = SimScore(age[i],age[j],sex[i],sex[j],loc[i],loc[j],his[i],his[j],
                                pts[i],pts[j],pns[i],pns[j], pms[i],pms[j],tnm[i],tnm[j])
            if adj[i,j] != 0:
                edge_list.append([i,j])
                edge_wight.append(adj[i,j])
    return adj, edge_list,edge_wight

In [10]:

def graph_bulider(all_data, start_cloumn = 13, event = "OS_Status", label = "OS_Month"):

    # save the labels
    norm_label_sh = all_data[label]
    # norm_label = (final_data['OS_Month']-np.min(final_data['OS_Month']))/(np.max(final_data['OS_Month'])-np.min(final_data['OS_Month']))
    labels_sh = torch.from_numpy(norm_label_sh.to_numpy())
    
    events_sh = torch.from_numpy(all_data[event].to_numpy())
    
    adj_sh, edge_list_sh, edge_wight_sh = adj_matrix(all_data)
    print("the number of nodes in this graph:",len(norm_label_sh))
    print("the number of edges in this graph:",len(edge_list_sh))
    print("Number of average degree: ",len(edge_list_sh)/len(norm_label_sh) )
    
    # build graph struture data
    g_sh = dgl.DGLGraph()
    g_sh.add_nodes(len(labels_sh))
    # add nodes
    # node_feature = (all_data.iloc[:, 15:]-all_data.iloc[:, 15:].min())/(all_data.iloc[:, 15:].max()- all_data.iloc[:, 15:].min())
    node_feature_sh = all_data.iloc[:, start_cloumn:]
    # print(node_feature)
    node_feature_norm_sh = node_feature_sh.to_numpy()
    g_sh.ndata['h'] = torch.from_numpy(node_feature_norm_sh).float()
    g_sh.ndata['event'] = events_sh
    g_sh.ndata['label'] = labels_sh
    g_sh.ndata
    # g.adj = adj
    # add edges
    src, dst = tuple(zip(*edge_list_sh))
    g_sh.add_edges(src, dst)
    # add edge weight
    edge_wight_sh = np.array(edge_wight_sh)
    g_sh.edata['w'] = torch.from_numpy(edge_wight_sh).float()
    return adj_sh, g_sh

# Network and Loss

In [11]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, dropout=0, activation = None,aggregator_type='mean'):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = SAGEConv(in_feats=hid_feats, out_feats=hid_feats, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.conv2 = SAGEConv(in_feats=hid_feats, out_feats= out_feats, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.fc2 = nn.Linear(out_feats, 1) 
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
        h = self.fc1(inputs)
        h = self.conv1(graph, h, w_input)
        h = self.conv2(graph,h)
#         print(h.size())
#         output=F.relu(self.fc2(h))
        output= self.fc2(h)

        return output
    
class TAG(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, activation = F.softmax):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = TAGConv(in_feats=hid_feats, out_feats= hid_feats, activation=activation)
        self.conv2 = TAGConv(in_feats=hid_feats, out_feats= out_feats,  activation=activation)
        self.fc2 = nn.Linear(out_feats, 1) 
        
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
        h= self.fc1(inputs)
        h = self.conv1(graph, h)
        h = self.conv2(graph,h)
#         output=F.relu(self.fc2(h))
        h=self.fc2(h)
        
        return h



    
class GCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, activation = F.softmax, norm ="both"):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = GraphConv(in_feats=hid_feats, out_feats= 32, activation=activation, norm=norm)
        self.conv2 = GraphConv(in_feats=32, out_feats= out_feats,  activation=activation, norm=norm)
        self.fc2 = nn.Linear(out_feats, 1) 
        
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
        h= self.fc1(inputs)
        h = self.conv1(graph, h)
        h = self.conv2(graph,h)
#         output=F.relu(self.fc2(h))
        h=self.fc2(h)
        
        return h
    
    
class SAGE1L(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, dropout=0, activation = None,aggregator_type='mean'):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = SAGEConv(in_feats=hid_feats, out_feats=out_feats, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.fc2 = nn.Linear(out_feats, 1) 
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
        h = self.fc1(inputs)
        h = self.conv1(graph, h, w_input)

        output= self.fc2(h)

        return output
    


In [12]:
class Regularization(object):
    def __init__(self, order, weight_decay):
        ''' The initialization of Regularization class
        :param order: (int) norm order number
        :param weight_decay: (float) weight decay rate
        '''
        super(Regularization, self).__init__()
        self.order = order
        self.weight_decay = weight_decay

    def __call__(self, model):
        ''' Performs calculates regularization(self.order) loss for model.
        :param model: (torch.nn.Module object)
        :return reg_loss: (torch.Tensor) the regularization(self.order) loss
        '''
        reg_loss = 0
        for name, w in model.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(w, p=self.order)
        reg_loss = self.weight_decay * reg_loss
        return reg_loss

    
class NegativeLogLikelihood(nn.Module):
    def __init__(self, l2_reg, device):
        super(NegativeLogLikelihood, self).__init__()
        self.L2_reg = l2_reg
        self.device = device
        self.reg = Regularization(order=2, weight_decay=self.L2_reg)

    def forward(self, risk_pred, y, e, model):
        mask = torch.ones(y.shape[0], y.shape[0]).to(self.device)
        mask[(y.T - y) > 0] = 0
        log_loss = torch.exp(risk_pred) * mask
        log_loss = torch.sum(log_loss, dim=0) / torch.sum(mask, dim=0)
        log_loss = torch.log(log_loss).reshape(-1, 1)
        neg_log_loss = -torch.sum((risk_pred-log_loss) * e) / torch.sum(e)
        l2_loss = self.reg(model)
        return neg_log_loss + l2_loss

In [13]:
adj_all, g_all = graph_bulider(final_data)
adj_sh, g_sh = graph_bulider(all_data)
# g_sh = dgl.node_subgraph(g_all, list(range(len(all_data))))
g_external = dgl.node_subgraph(g_all, list(range(len(all_data),len(final_data))))
# g_val = graph_bulider(all_data)

the number of nodes in this graph: 1864
the number of edges in this graph: 387396
Number of average degree:  207.8304721030043


/home/jielian/anaconda3/envs/gnn/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


the number of nodes in this graph: 1705
the number of edges in this graph: 330049
Number of average degree:  193.57712609970673


# Training

In [20]:
def train(g, g_all, model,device, save_dic, idx_train,idx_val, idx_test, total_epoch=100, patience=5, lr=0.001, reg_l2=0, weight_decay=0.0001):
    model_name = save_dic['model']+str(save_dic['hid_feats'])+str(save_dic['out_feats'])+str(save_dic['reg_l2'])+save_dic["aggregator_type"]
    optimizer = torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    best_cindex = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.5, patience=patience, min_lr = 0.0001, verbose=True)
    criterion = NegativeLogLikelihood(reg_l2, device).to(device) 
    model = model.to(device) 
    features = g.ndata['h'].to(device) 
    e_feature = g.edata['w'].to(device) 
    labels = g.ndata['label'].to(device) 
    events = g.ndata['event'].to(device) 
    g_all= g_all.to(device)  
    g = g.to(device) 
    t_total = time.time()
    with tqdm(range(total_epoch)) as t:
        for epoch in t:
            t.set_description('Epoch %d' % epoch)
            start = time.time()
            model.train()
            optimizer.zero_grad()
            output = model(g, features,e_feature)
            # Compute loss
            # Note that you should only compute the losses of the nodes in the training set.
            loss_train = criterion(output[idx_train], labels[idx_train],events[idx_train], model).clone()
            auc_train = c_index(-output[idx_train], labels[idx_train],events[idx_train])
            
            loss_train.backward(retain_graph=True)
            optimizer.step()
            
            model.eval()
            val_output = model(g, features,e_feature)
            loss_val = criterion(val_output[idx_val], labels[idx_val],events[idx_val], model).clone()
            scheduler.step(loss_val)
            
            auc_val = c_index(-val_output[idx_val], labels[idx_val],events[idx_val])
            auc_test = c_index(-val_output[idx_test], labels[idx_test], events[idx_test])
            exter_val_output = model(g_all, g_all.ndata['h'],g_all.edata['w'])
            
            if auc_test>best_cindex:
                print("Curent best Test AUC:",auc_test)
                best_cindex = auc_test.item()
                print("Its' val AUC :", auc_val)
                auc_external = c_index(-exter_val_output, g_all.ndata['label'] ,g_all.ndata['event'])
                print("Its external  AUC:", auc_external) 
                
                if best_cindex > 0.67 and auc_external>0.60:
                    torch.save(model.state_dict(), os.path.join(save_dic['save_path'], 
                                "{}_ep{}_val{}_test{}_exte{}.pth.gz".format(model_name,epoch, np.around(auc_val,3),np.around(auc_test,3),np.around(auc_external,3))))
                    result_path = os.path.join(save_dic['save_path'], "{}_ep{}_val{}_test{}_exte{}.npy".format(model_name,epoch, np.around(auc_val,3),np.around(auc_test,3),np.around(auc_external,3)))
                    output_cpu = val_output.cpu().detach().numpy()
                    val_cpu = exter_val_output.cpu().detach().numpy()
                    np.save(result_path, [output_cpu, val_cpu])
                    

            t.set_postfix(
                  {"train_loss":loss_train.item(), "val_loss":loss_val.item(),
                  "train_cindex":auc_train.item(), "val_auc":auc_val.item(),
                "lr":optimizer.param_groups[0]['lr']}) 

In [18]:
 save_dic = {"model":"SAGE", 
                "hid_feats":256, 
                'out_feats':12, 
                'reg_l2':0,
                "aggregator_type":'mean',
                "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/resnet/"}

model = SAGE(g_sh.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                       activation = F.leaky_relu, aggregator_type=save_dic['aggregator_type'])
train(g_sh, g_external, model, device, save_dic, idx_train, idx_val, idx_test, 50, patience=5, reg_l2=save_dic["reg_l2"])

Epoch 6:  10%| | 5/50 [00:00<00:01, 25.05it/s, train_loss=4.08, val_loss=0.421, 

Curent best Test AUC: 0.5323731997495303
Its' val AUC : 0.5608264238140547
Its external  AUC: 0.4246100519930676
Curent best Test AUC: 0.5849718221665623
Its' val AUC : 0.6344977572380046
Its external  AUC: 0.4755002363321254


Epoch 17:  34%|▎| 17/50 [00:00<00:00, 33.90it/s, train_loss=1.21, val_loss=0.101

Curent best Test AUC: 0.5912335629304947
Its' val AUC : 0.6085360880793802
Its external  AUC: 0.465574287064755
Curent best Test AUC: 0.6006261740763932
Its' val AUC : 0.627157808889493
Its external  AUC: 0.48259020009453285


Epoch 43:  78%|▊| 39/50 [00:01<00:00, 38.98it/s, train_loss=0.036, val_loss=0.01

Epoch    35: reducing learning rate of group 0 to 5.0000e-04.
Curent best Test AUC: 0.6033813400125235
Its' val AUC : 0.623759684654071
Its external  AUC: 0.5085867338900267


Epoch 49: 100%|█| 50/50 [00:01<00:00, 36.38it/s, train_loss=0.0472, val_loss=0.0

Epoch    44: reducing learning rate of group 0 to 2.5000e-04.
Curent best Test AUC: 0.6170319348778961
Its' val AUC : 0.582302568981922
Its external  AUC: 0.5161493619032614


In [23]:
for i in range(100):
    save_dic = {"model":"SAGE", 
                "hid_feats":256, 
                'out_feats':12, 
                'reg_l2':0,
                "aggregator_type":'mean',
                "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/resnet/"}

    model = SAGE(g_sh.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                           activation = F.leaky_relu, aggregator_type=save_dic['aggregator_type'])
    train(g_sh, g_external, model, device, save_dic, idx_train, idx_val, idx_test, 50, patience=10, reg_l2=save_dic["reg_l2"])

Epoch 5:  10%| | 5/50 [00:00<00:01, 22.51it/s, train_loss=0.364, val_loss=0.359,

Curent best Test AUC: 0.5278647463994991
Its' val AUC : 0.6180508359385619
Its external  AUC: 0.5796439262643769
Curent best Test AUC: 0.5343769567939888
Its' val AUC : 0.5012912872094604
Its external  AUC: 0.4224042854892075
Curent best Test AUC: 0.5624295554164057
Its' val AUC : 0.6506728285986135
Its external  AUC: 0.5838979045218213


Epoch 12:  24%|▏| 12/50 [00:00<00:01, 27.95it/s, train_loss=0.243, val_loss=0.03

Curent best Test AUC: 0.5884783969943644
Its' val AUC : 0.5938561913823569
Its external  AUC: 0.5764928312588624


Epoch 33:  64%|▋| 32/50 [00:01<00:00, 32.69it/s, train_loss=0.0413, val_loss=0.0

...

Curent best Test AUC: 0.43018159048215404
Its' val AUC : 0.3421231480222917
Its external  AUC: 0.500551441625965
Curent best Test AUC: 0.6364433312460864
Its' val AUC : 0.551175750985456
Its external  AUC: 0.553647392468883


Epoch 21:  40%|▍| 20/50 [00:00<00:00, 35.83it/s, train_loss=0.0253, val_loss=0.0

Epoch    15: reducing learning rate of group 0 to 5.0000e-04.


Epoch 49: 100%|█| 50/50 [00:01<00:00, 36.25it/s, train_loss=0.00159, val_loss=0.

Epoch    50: reducing learning rate of group 0 to 2.5000e-04.


# Testing!

In [24]:
g = g_sh
#test!
save_dic = {"model":"SAGE", 
            "hid_feats":256, 
            'out_feats':12, 
            "aggregator_type":'mean',
            "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/resnet/"}
features = g.ndata['h']
e_feature = g.edata['w']
labels = g.ndata['label']
events = g.ndata['event']
model = SAGE(g.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                   activation = F.relu, aggregator_type=save_dic['aggregator_type'])
pre_train='SAGE256120mean_ep20_val0.713_test0.706_exte0.604.pth.gz'
state_dict=torch.load(os.path.join(save_dic["save_path"]+pre_train), map_location='cpu')
model.load_state_dict(state_dict)
model.eval()
outputs = model.forward(g, features,e_feature)
auc_train = c_index(-outputs[idx_train], g_all.ndata['label'][idx_train],g_all.ndata['event'][idx_train])
print("Training:", auc_train)
auc_val = c_index(-outputs[idx_val], g_all.ndata['label'][idx_val],g_all.ndata['event'][idx_val])
print("Validation:", auc_val)
auc_test = c_index(-outputs[idx_test], g_all.ndata['label'][idx_test],g_all.ndata['event'][idx_test])
print("Testing:", auc_test)
outputs_val= model.forward(g_external, g_external.ndata['h'],g_external.edata['w'])
Exter_test = c_index(-outputs_val, g_all.ndata['label'][idx_external_val],g_all.ndata['event'][idx_external_val])
print("External:",Exter_test)
Exter_test_val = c_index(-outputs_val, g_external.ndata['label'],g_external.ndata['event'])
print("External:",Exter_test_val)


Training: 0.6025972673428319
Validation: 0.6620905260296317
Testing: 0.6351909830932999
External: 0.4813297620923271
External: 0.4813297620923271


In [25]:
result1, result2 = np.load("logs/resnet/SAGE256120mean_ep20_val0.713_test0.706_exte0.604.npy", allow_pickle=True)
auc_test = c_index(-result1[idx_test], g_all.ndata['label'][idx_test],g_all.ndata['event'][idx_test])
print(auc_test)
print(roc_auc_score(g_all.ndata['event'][idx_test],result1[idx_test]))

print(c_index(-result2, g_external.ndata['label'],g_external.ndata['event']))
print(roc_auc_score(g_external.ndata['event'],result2))

0.7061991233562931
0.7311726734803657
0.6040649125571136
0.6267629858961128


In [26]:
from lifelines.utils import concordance_index
risk_score = []
for i in range(len(outputs)):
    risk_score.append(outputs[i].item())
patient_info['risk'] = risk_score
test = patient_info.iloc[test_id,:]
print(concordance_index(test['OS_Month'], -test['risk'], test['OS_Status']))

exter_risk_score = []
for i in range(len(outputs_val)):
    exter_risk_score.append(outputs_val[i].item())
external_info['risk'] = exter_risk_score
print(concordance_index(external_info['OS_Month'], -external_info['risk'], external_info['OS_Status']))

0.6351909830932999
0.4813297620923271


/tmp/ipykernel_6264/3655968121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_info['risk'] = risk_score


In [26]:
test_result_path = "/home/jielian/lung-graph-project/Tumor_tranformer/logs/results/"
test_result_name = "ResNet"+pre_train[:-7]+".csv"
test.to_csv(test_result_path+test_result_name, index = False)

In [27]:
EXTtest_result_path = "/home/jielian/lung-graph-project/Tumor_tranformer/logs/results/"
EXTtest_result_name = "ResNet"+pre_train[:-7]+"_EXTERNAL.csv"
external_info.to_csv(EXTtest_result_path+EXTtest_result_name, index = False)

In [28]:
print("========AUC=========")
print("OS on Testing Set:", roc_auc_score(test['OS_Status'], test['risk']))
print("OS on External Set:",roc_auc_score(external_info['OS_Status'], external_info['risk']))
print("RFS on Testing Set:",roc_auc_score(test['RFS_Status'], test['risk']))
print("RFS on External Set:",roc_auc_score(external_info['RFS_Status'], external_info['risk']))

========AUC=========
OS on Testing Set: 0.6754975793437332
OS on External Set: 0.6314069487444101
RFS on Testing Set: 0.6250275148580233
RFS on External Set: 0.7073874567141207
